In [4]:
from etl import FileMigrator
import os
import psycopg2

In [6]:
from etl import CongressionalAPI

In [8]:
api = CongressionalAPI(os.environ["PRO_PUBLICA_API_KEY"])

In [14]:
mems = api.fetch_members({"house": [str(i) for i in range(102, 116)], "senate": [str(i) for i in range(80, 116)]})

/Users/benellis/Projects/congress/etl.py:267: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat(all_members)


In [37]:
dtype_mapper = {
    "object": "TEXT",
    'float64': "DECIMAL",
    'bool': 'BOOLEAN'
}
print("CREATE TABLE IF NOT EXISTS members(")
for i in mems.columns:
    if i == 'date_of_birth':
        print(i, 'DATE')
    else:
        print(i, dtype_mapper[str(mems[i].dtype)] + ",")
print("PRIMARY KEY (last_name, first_name, congress, chamber)")
print(")")

CREATE TABLE IF NOT EXISTS members(
api_uri TEXT,
at_large TEXT,
chamber TEXT,
congress TEXT,
contact_form TEXT,
cook_pvi TEXT,
crp_id TEXT,
cspan_id TEXT,
date_of_birth DATE
district TEXT,
dw_nominate TEXT,
facebook_account TEXT,
fax TEXT,
fec_candidate_id TEXT,
first_name TEXT,
gender TEXT,
geoid TEXT,
google_entity_id TEXT,
govtrack_id TEXT,
icpsr_id TEXT,
id TEXT,
ideal_point TEXT,
in_office BOOLEAN,
last_name TEXT,
last_updated TEXT,
leadership_role TEXT,
lis_id TEXT,
middle_name TEXT,
missed_votes TEXT,
missed_votes_pct DECIMAL,
next_election TEXT,
ocd_id TEXT,
office TEXT,
party TEXT,
phone TEXT,
rss_url TEXT,
senate_class TEXT,
seniority TEXT,
short_title TEXT,
state TEXT,
state_rank TEXT,
suffix TEXT,
title TEXT,
total_present TEXT,
total_votes TEXT,
twitter_account TEXT,
url TEXT,
votes_against_party_pct DECIMAL,
votes_with_party_pct DECIMAL,
votesmart_id TEXT,
youtube_account TEXT,
PRIMARY KEY (last_name, first_name, congress, chamber)
)


In [63]:
def prep_for_psql(d):
    vals = []
    for c in ['api_uri', 'at_large', 'chamber', 'congress', 'contact_form', 'cook_pvi', 
              'crp_id', 'cspan_id', 'date_of_birth', 'district', 'dw_nominate', 'facebook_account', 
              'fax', 'fec_candidate_id', 'first_name', 'gender', 'geoid', 'google_entity_id', 
              'govtrack_id', 'icpsr_id', 'id', 'ideal_point', 'in_office', 'last_name', 'last_updated', 
              'leadership_role', 'lis_id', 'middle_name', 'missed_votes', 'missed_votes_pct', 
              'next_election', 'ocd_id', 'office', 'party', 'phone', 'rss_url', 'senate_class', 
              'seniority', 'short_title', 'state', 'state_rank', 'suffix', 'title', 'total_present', 
              'total_votes', 'twitter_account', 'url', 'votes_against_party_pct', 'votes_with_party_pct', 
              'votesmart_id', 'youtube_account']:
        if c == 'date_of_birth' and d[c] == '':
            d[c] = '1900-01-01'
        vals.append(d[c])
    return vals


def df_to_psql(df):
    for row in df.to_dict(orient="records"):
        q = \
        """
        INSERT INTO members 
            (api_uri, at_large, chamber, congress, contact_form, cook_pvi, crp_id,
             cspan_id, date_of_birth, district, dw_nominate, facebook_account,
             fax, fec_candidate_id, first_name, gender, geoid, google_entity_id,
             govtrack_id, icpsr_id, id, ideal_point, in_office, last_name, last_updated, 
             leadership_role, lis_id, middle_name, missed_votes, missed_votes_pct,
             next_election, ocd_id, office, party, phone, rss_url, senate_class,
             seniority, short_title, state, state_rank, suffix, title, total_present,
             total_votes, twitter_account, url, votes_against_party_pct, votes_with_party_pct,
             votesmart_id, youtube_account)
        VALUES
            (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,
             %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
        ON CONFLICT ON CONSTRAINT members_pkey DO NOTHING
        """
        conn = psycopg2.connect(
            user=os.environ.get('PSQL_USER', 'benellis'), 
            password=os.environ.get('PSQL_PASSWORD', 'daisythedog'), 
            database='congress', 
            host=os.environ.get('PSQL_HOST', 'localhost'), 
            port=os.environ.get('PSQL_PORT', '5432')
        )
        cursor = conn.cursor()
        command = cursor.mogrify(q, (prep_for_psql(row)))
        cursor.execute(command)
        conn.commit()
        conn.close()

In [64]:
df_to_psql(mems)